In [ ]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(source("tree_term_climb.R"))
suppressPackageStartupMessages(library(beaveR))

saveDir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts/environment/real_datasets/GSE100505_EDL_MAST_subsamp/mode=gcbias/post_type=gibbs_nreps=100_tf=100/DE"
load(file.path(saveDir, "tseCons.RData"))

In [ ]:
system.file('extdata/brain_sim_nodtu_small_example', package='beaveR')

In [ ]:
yAll <- computeSizeFactors(tseCons)
yAll <- scaleInfReps(yAll)
yAll <- labelKeep(yAll)
set.seed(1)
yAll <- swish(yAll, x = "condition")

In [ ]:
treeCons <- rowTree(tseCons)
signs <- computeSign(yAll, "condition")
print(table(signs))
levels <- node.depth(treeCons, 2)
levels <- ifelse(levels > 4, 5, levels)
print(table(levels))
print(table(signs, levels))

In [6]:
mcols(yAll)[2,]

DataFrame with 1 row and 8 columns
                     meanInfRV log10mean      keep      stat    log2FC
                     <numeric> <numeric> <logical> <numeric> <numeric>
ENSMUST00000057438.6 0.0744576   2.53817      TRUE      17.7  0.748388
                          pvalue    locfdr    qvalue
                       <numeric> <numeric> <numeric>
ENSMUST00000057438.6 0.000702587 0.0186326 0.0117192

In [2]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(source("tree_term_climb.R"))
suppressPackageStartupMessages(library(beaveR))

In [5]:
sI <- function (y, lengthCorrect = TRUE, meanDepth = NULL, sfFun = NULL, 
    minCount = 10, minN = 3, saveMeanScaled = FALSE, quiet = FALSE) 
{
    if (!interactive()) {
        quiet <- TRUE
    }
    if (!is.null(metadata(y)$infRepsScaled)) {
        if (metadata(y)$infRepsScaled) 
            stop("inferential replicates already scaled")
    }
    infRepIdx <- grep("infRep", assayNames(y))
    infReps <- assays(y)[infRepIdx]
    sfs <- matrix(0, ncol=ncol(y), nrow=length(infReps))
    counts <- assays(y)[["counts"]]
    length <- assays(y)[["length"]]
    nreps <- length(infReps)
    if (is.null(meanDepth) & !is(sfFun, "numeric")) {
        meanDepth <- exp(mean(log(colSums(counts))))
    }
    
    means <- matrix(nrow = nrow(y), ncol = nreps)
    if (is.null(length)) {
        if (lengthCorrect) {
            if (!quiet) 
                message("not correcting for feature length (lengthCorrect=FALSE)")
        }
        lengthCorrect <- FALSE
    }
#     for (k in seq_len(nreps)) {
    for (k in seq(5)) {
        if (!quiet) 
            svMisc::progress(k, max.value = nreps, init = (k == 
                1), gui = FALSE)
        if (lengthCorrect) {
            length <- length/exp(rowMeans(log(length)))
            cts <- infReps[[k]]/length
        }
        else {
            cts <- infReps[[k]]
        }
        if (!is(sfFun, "numeric")) {
#             print(cts[1,1:5])
            print(dim(cts))
            print(paste(colSums(cts), meanDepth))
            cts <- t(t(cts)/colSums(cts)) * meanDepth
            use <- rowSums(infReps[[k]] >= minCount) >= minN
            print(cts[1,1:5])
#             print(sum(use))
            
            if (is.null(sfFun)) {
                loggeomeans <- rowMeans(log(cts[use, ]))
                sf <- apply(cts[use, ], 2, function(s) {
                  exp(median((log(s) - loggeomeans)[is.finite(loggeomeans)]))
                })
            }
            else if (is(sfFun, "function")) {
                sf <- sfFun(cts)
            }
        }
        else {
            sf <- sfFun
        }
        sfs[k,] <- sf
        infReps[[k]] <- t(t(cts)/sf)
        means[, k] <- rowMeans(infReps[[k]])
    }
    if (!quiet) 
        message("")
    assays(y)[grep("infRep", assayNames(y))] <- infReps
    mcols(y)$log10mean <- log10(rowMeans(means) + 1)
    metadata(y)$infRepsScaled <- TRUE
    if (saveMeanScaled) {
        infRepsArray <- abind::abind(as.list(infReps), along = 3)
        meanScaled <- apply(infRepsArray, 1:2, mean)
        assays(y)[["meanScaled"]] <- meanScaled
    }
    sfs
}


In [6]:
ss <- sI(y)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'metadata': object 'tse' not found
